In [109]:
import pandas as pd

#google cloud big query libraries
from google.cloud import bigquery
from google.oauth2 import service_account

if os.getlogin()=='cday':
    key_path = r"C:\Users\cday\tdm-scenarios-c90ba30c3c5d.json"
elif os.getlogin()=='bhereth':
    key_path = r"C:\Users\bhereth\tdm-scenarios-a85044dbbfd3.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
 
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [136]:
dfModeGroups = pd.DataFrame([
    ['Walk'     ,'1: Non-Motorized'         ],
    ['Bike'     ,'1: Non-Motorized'         ],
    ['Walk'     ,'1a: Walk'                 ],
    ['Bike'     ,'1b: Bike'                 ],
    ['Auto'     ,'2: Auto'                  ],
    ['SchoolBus','3: SchoolBus'             ],
    ['LCL'      ,'4: Transit'               ],
    ['COR'      ,'4: Transit'               ],
    ['EXP'      ,'4: Transit'               ],
    ['BRT'      ,'4: Transit'               ],
    ['LRT'      ,'4: Transit'               ],
    ['CRT'      ,'4: Transit'               ],
    ['LCL'      ,'4a: Local Bus'            ],
    ['COR'      ,'4b: Core Bus'             ],
    ['EXP'      ,'4c: Express Bus'          ],
    ['BRT'      ,'4d: Bus-Rapid Transit'    ],
    ['LRT'      ,'4e: Light-Rail Transit'   ],
    ['CRT'      ,'4f: Commuter-Rail Transit']
],columns=('MODE','modeGroup'))
dfModeGroups

,MODE,modeGroup
0,Walk,1: Non-Motorized
1,Bike,1: Non-Motorized
2,Walk,1a: Walk
3,Bike,1b: Bike
4,Auto,2: Auto
5,SchoolBus,3: SchoolBus
6,LCL,4: Transit
7,COR,4: Transit
8,EXP,4: Transit
9,BRT,4: Transit


In [137]:
dfTransitSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.transit_share").to_dataframe()
display(dfTransitSummary)

dfScenarios = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios").to_dataframe()
display(dfScenarios)

dfTransitSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfTransitSummary,on='scenarioID')
display(dfTransitSummaryWithScenarioData)

,scenarioID,PERIOD,TRIPPURP,MODE,TRIPS
0,1,Ok,HBC,Walk,0
1,1,Ok,HBC,Bike,0
2,1,Ok,HBC,Auto,0
3,1,Ok,HBC,SchoolBus,0
4,1,Ok,HBC,LCL,0
...,...,...,...,...,...
2995,35,Pk,NHB,COR,29
2996,35,Pk,NHB,BRT,659
2997,35,Pk,NHB,EXP,5
2998,35,Pk,NHB,LRT,3747


,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear
0,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019
1,2,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2030,RTP,2030
2,3,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2040,RTP,2040
3,4,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2050,RTP,2050
4,5,v832,WF TDM v8.3.2 - 2022-02-04a,RTP_2050,RTP,2050
5,6,v832,WF TDM v8.3.2 - 2022-02-04a,TIP_2024,TIP,2024
6,7,v832,WF TDM v8.3.2 - 2022-09-21,BY_2019,BY,2019
7,8,v832,WF TDM v8.3.2 - 2022-09-21,BY_2023,BY,2023
8,9,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2032,Needs,2032
9,10,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2042,Needs,2042


,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear,PERIOD,TRIPPURP,MODE,TRIPS
0,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBC,Walk,0
1,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBC,Bike,0
2,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBC,Auto,0
3,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBC,SchoolBus,0
4,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBC,LCL,0
...,...,...,...,...,...,...,...,...,...,...
2995,35,v9,WF TDM v9.0 - 2022-12-19,TIP_2028,TIP,2028,Pk,NHB,COR,29
2996,35,v9,WF TDM v9.0 - 2022-12-19,TIP_2028,TIP,2028,Pk,NHB,BRT,659
2997,35,v9,WF TDM v9.0 - 2022-12-19,TIP_2028,TIP,2028,Pk,NHB,EXP,5
2998,35,v9,WF TDM v9.0 - 2022-12-19,TIP_2028,TIP,2028,Pk,NHB,LRT,3747


In [138]:
dfTransitSummaryPlotData = pd.DataFrame.merge(dfTransitSummaryWithScenarioData,dfModeGroups,on='MODE')
dfTransitSummaryPlotData

,scenarioID,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear,PERIOD,TRIPPURP,MODE,TRIPS,modeGroup
0,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBC,Walk,0,1: Non-Motorized
1,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBC,Walk,0,1a: Walk
2,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBO,Walk,296767,1: Non-Motorized
3,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBO,Walk,296767,1a: Walk
4,1,v832,WF TDM v8.3.2 - 2022-02-04a,BY_2019,BY,2019,Ok,HBS,Walk,76704,1: Non-Motorized
...,...,...,...,...,...,...,...,...,...,...,...
5395,35,v9,WF TDM v9.0 - 2022-12-19,TIP_2028,TIP,2028,Pk,HBS,CRT,0,4f: Commuter-Rail Transit
5396,35,v9,WF TDM v9.0 - 2022-12-19,TIP_2028,TIP,2028,Pk,HBW,CRT,12291,4: Transit
5397,35,v9,WF TDM v9.0 - 2022-12-19,TIP_2028,TIP,2028,Pk,HBW,CRT,12291,4f: Commuter-Rail Transit
5398,35,v9,WF TDM v9.0 - 2022-12-19,TIP_2028,TIP,2028,Pk,NHB,CRT,691,4: Transit


In [139]:
#PLOTTING FUNCTION

def update_plot(tdmVersionsWithDate, scenarioGroups, modeGroups, trippurps, periods):

    data = []

    for v in tdmVersionsWithDate:
        for g in scenarioGroups:
            for m in modeGroups:

                # data for plotting from filtered dataframe}
                plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin(['BY',g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))]

                #display(plotdata)

                plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

                # fill any NaN values with zeros
                plotdata = plotdata.fillna(0)

                display(plotdata)

                xplot = plotdata['scenarioYear']
                yplot = plotdata['TRIPS'       ]

                trace1 = go.Scatter(
                    x=xplot,
                    y=yplot,
                    mode='lines',
                    name= v + ' ' + g + ' ' + m#,
    #                line=dict(
    #                    shape='spline'
    #                )
                )
                data.append(trace1)


    layout = go.Layout(
        title='Transit Trips',
        yaxis=dict(
            title='Trips',
            range=(0,100000)
        ),
        xaxis=dict(
            title='Year',
            range=(2019,2050)
        )#,
        #width=1600,
        #height=450
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [140]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstTdmVersionWithDate = dfTransitSummaryPlotData['tdmVersionWithDate'].unique().tolist()
lstScenarioGroups     = dfTransitSummaryPlotData['scenarioGroup'     ].unique().tolist()
lstModes              = dfTransitSummaryPlotData['modeGroup'         ].unique().tolist()
lstTripPurp           = dfTransitSummaryPlotData['TRIPPURP'          ].unique().tolist()
lstPeriod             = dfTransitSummaryPlotData['PERIOD'            ].unique().tolist()

selectTdmVersionsWithDate = widgets.SelectMultiple(options=lstTdmVersionWithDate, value=(lstTdmVersionWithDate[1],), description='TDM Version'   )
selectScenarioGroups      = widgets.SelectMultiple(options=lstScenarioGroups    , value=('RTP'                   ,), description='Scenario Group')
selectModes               = widgets.SelectMultiple(options=lstModes             , value=('4: Transit'               ,), description='Mode Group')
selectTripPurps           = widgets.SelectMultiple(options=lstTripPurp          , value=(lstTripPurp[0]          ,), description='Purpose'       )
selectPeriods             = widgets.SelectMultiple(options=lstPeriod            , value=(lstPeriod[1]            ,), description='Periods'       )

#custom extents
#xmin = widgets.Text(value='0'  , description='X Min')
#xmax = widgets.Text(value='160' , description='X Max')
#ymin = widgets.Text(value='0'  , description='Y Min')
#ymax = widgets.Text(value='.10', description='Y Max')

widgets.interactive(update_plot, tdmVersionsWithDate=selectTdmVersionsWithDate, scenarioGroups=selectScenarioGroups, modeGroups=selectModes, trippurps=selectTripPurps , periods=selectPeriods)

#ADD RAW OBSERVED CURVES

TraitError: Invalid selection: value not found